In [ ]:
import itertools
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from enum import Enum
# 輸入資料型態
from sklearn.utils import shuffle
import os
# 資料類別
class DataType(Enum):
    ALL_DATA = 'all data'       # 所有資料
    TRAIN = 'train data'        # 只有訓練資料
    TEST = 'test data'          # 只有測試資料
    CUSTOM = 'custom data'      # 原先設計好之資料
    SMALL = 'small dataset'     # 小型資料集

# 輸入資料
class Data(object):
    def __init__(self, data_type=DataType.ALL_DATA,
                 img_width=224, img_height=224,
                 split_data=False, filename='small_data', num_classes=6,
                 data_generator=True):
        print("- create data instance")
        try:
            print("\t- set data type ... ", end="")
            self.data_type=data_type
            self.num_classes=num_classes
            self.filename = filename
            print("{}".format(data_type.name))
        except:
            print("")
        try:
            print('\t- set X and Y ... ', end="")
            self.x = np.load('E:\\交接\\1_賴念祥code\\賴念祥code\\Researchyes\\Research\\Dataset\\' + filename + '_x.npy')
            self.y = np.load('E:\\交接\\1_賴念祥code\\賴念祥code\\Researchyes\\Research\\Dataset\\' + filename + '_y.npy')
            print("OK")
        except:
            print("")
        try:
            print("\t- set image shape ... ", end="")
            self.img_width=img_width
            self.img_height=img_height
            print("OK")
        except:
            print("image shape setting error")
        self.split_data=split_data
        try:
            print("\t- start data pre process ... ", end="")
            self.data_generator = data_generator
            self.pre_process()
           # print("OK")
        except Exception as e:
            print(e)


    # 資料預處理
    def pre_process(self, test_size=0.2, val_size=0.1, filename=None):
        x, y = shuffle(self.x, self.y)
        x = x.reshape(x.shape[0], self.img_width, self.img_height, 1)
        if (self.data_type == DataType.ALL_DATA):
            train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=test_size, random_state=25)
            train_x, val_x, train_y, val_y = train_test_split(x, y, test_size=val_size, random_state=25)
        if (self.data_type == DataType.TRAIN):
            train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=test_size, random_state=25)
            val_x = test_x
            val_y = test_y
        if (self.data_type == DataType.TEST or self.data_type == DataType.SMALL):
            train_x = x
            train_y = y
            test_x = x
            test_y = y
            val_x = x
            val_y = y

        if (self.data_type == DataType.CUSTOM):
            self.train_x = self.arr_reshape(x)
            self.train_y = y
            self.test_x, self.test_y = shuffle(np.load('E:\\交接\\1_賴念祥code\\賴念祥code\\Researchyes\\Research\\Dataset\\{}_test_x.npy'.format(filename)), np.load('Dataset\\{}_test_y.npy'.format(filename)))
            self.val_x, self.val_y = shuffle(np.load('E:\\交接\\1_賴念祥code\\賴念祥code\\Researchyes\\Research\\Dataset\\{}_val_x.npy'.format(filename)), np.load('Dataset\\{}_val_y.npy'.format(filename)))
            self.test_x = self.arr_reshape(self.test_x)
            self.val_x = self.arr_reshape(self.val_x)

        self.train_x = train_x
        self.train_y = train_y
        self.val_x = val_x
        self.val_y = val_y
        self.test_x = test_x
        self.test_y = test_y
        self.one_hot_encoding()
        self.save_data()

    # 對輸出值進行 one hot encodcing
    def one_hot_encoding(self):
        try:
            #pass
            self.train_y = np_utils.to_categorical(self.train_y, self.num_classes).astype('int')
            self.test_y = np_utils.to_categorical(self.test_y, self.num_classes).astype('int')
            self.val_y = np_utils.to_categorical(self.val_y, self.num_classes).astype('int')
            #print("\n\none_hot_encoding done.\n\n")
        except:
            pass

    # 對 numpy 進行矩陣重組
    def arr_reshape(self, arr):
        return arr.reshape(arr.shape[0], self.img_width, self.img_height, 1)

    # 靜態取用矩陣重組
    @classmethod
    def static_arr_reshape(self, arr, img_width=224, img_height=224):
        return arr.reshape(arr.shape[0], img_width, img_height, 1)

    # 靜態取用 one hot encoding
    @classmethod
    def static_one_hot_encoding(self, y=None, classes=6):
        return np_utils.to_categorical(y, classes).astype('int')

    # 資料擴增
    def generator(self, batch_size):
        # train_datagen = ImageDataGenerator(horizontal_flip=True, rotation_range=10, zoom_range=0.2)
        # train_datagen = ImageDataGenerator(horizontal_flip=True, rotation_range=10)
        if self.data_generator:
            train_datagen = ImageDataGenerator(horizontal_flip=True, rotation_range=10, zoom_range=0.2)
            # train_datagen = ImageDataGenerator(horizontal_flip=True, rotation_range=10)
            train_datagen.fit(self.train_x)
            train_generator = train_datagen.flow(self.train_x, self.train_y, batch_size=batch_size)
            return train_generator
        else:
            train_datagen = ImageDataGenerator()
            train_datagen.fit(self.train_x)
            train_generator = train_datagen.flow(self.train_x, self.train_y, batch_size=batch_size)
            return train_generator

    # 資料儲存
    def save_data(self):
        np.save('D:\\NianXiang_File\\Research\\Dataset\\' + self.filename + '_train_x.npy', self.train_x)
        np.save('D:\\NianXiang_File\\Research\\Dataset\\' + self.filename + '_train_y.npy', self.train_y)
        np.save('D:\\NianXiang_File\\Research\\Dataset\\' + self.filename + '_val_x.npy', self.val_x)
        np.save('D:\\NianXiang_File\\Research\\Dataset\\' + self.filename + '_val_y.npy', self.val_y)
        np.save('D:\\NianXiang_File\\Research\\Dataset\\' + self.filename + '_test_x.npy', self.test_x)
        np.save('D:\\NianXiang_File\\Research\\Dataset\\' + self.filename + '_test_y.npy', self.test_y)

    # 將圖片轉換成矩陣
    @classmethod
    def load_data(self, path='', filename=''):
        folders, x, y= [], [], []
        for root, dirs, files in os.walk(path, topdown=False):
            for name in dirs:
                folders.append(name)
        folders = folders[-7:]
        start_time = time.time()
        count = 0
        for fld in folders:
            index = folders.index(fld)
            for root, dirs, files in os.walk(path + '\\' + fld, topdown=False):
                for file in files:
                    filepath = root + '\\' + file
                    img = cv2.imread(filepath)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    resized = cv2.resize(img, (224, 224), cv2.INTER_LANCZOS4)
                    y.append(index)
                    x.append(resized)
        np.save('{}_X.npy'.format(filename), x)
        np.save('{}_Y.npy'.format(filename), y)



    # 預測圖片轉換矩陣
    @classmethod
    def load_train_data(self, path="D:\\NianXiang_File\\CNN\\Dataset\\small_dataset\\NEW", filename=""):
        x = []
        y = []
        folders = []
        for root, dirs, files in os.walk(path, topdown=False):
            for name in dirs:
                folders.append(name)
        folders = folders[-7:]
        start_time = time.time()
        count = 0
        for fld in folders:
            index = folders.index(fld)
            print('Loading {} files (Index: {})'.format(fld, index))
            for root, dirs, files in os.walk(path + '\\' + fld, topdown=False):
                for file in files:
                    print(file)
                    if (file == '1.jpg'):
                        pass
                    else:
                        filepath = root + '\\' + file
                        img = cv2.imread(filepath)
                        try:
                            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                            resized = cv2.resize(img, (224, 224), cv2.INTER_LANCZOS4)
                            y.append(index)
                            x.append(resized)
                            count = count + 1
                        except:
                            pass

                            # file = Data.static_load_image(ffile)
        np.save('{}_x.npy'.format(filename), x)
        np.save('{}_y.npy'.format(filename), y)
        print("Complete")
    # 預測圖片轉換矩陣
    @classmethod
    def load_predict_data(self, path="", filename=""):
        x = []
        for root, dirs, files in os.walk(path, topdown=False):
            for file in files:
                filepath = root + '\\' + file
                img = cv2.imread(filepath)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                resized = cv2.resize(img, (224, 224), cv2.INTER_LANCZOS4)
                x.append(resized)
        np.save('{}_predict_X.npy'.format(filename), x)
    # 將圖片抓取臉部後轉換成矩陣
    @classmethod
    def get_data_from_path(self, path='D:\\NianXiang_File\\CNN\\Dataset\\LE', file_name='BE'):
        # data = np.load('predict_y_nor.npy')
        output = {}
        x = []
        y = []
        folders = []
        for root, dirs, files in os.walk(path, topdown=False):
            for name in dirs:
                folders.append(name)
        folders = folders[-7:]
        start_time = time.time()
        count = 0
        for fld in folders:
            index = folders.index(fld)
            print('Loading {} files (Index: {})'.format(fld, index))
            for root, dirs, files in os.walk(path + '\\' + fld, topdown=False):
                for file in files:
                    print(file)
                    if (file == '1.jpg'):
                        pass
                    else:
                        filepath = root + '\\' + file
                        detector = dlib.get_frontal_face_detector()
                        img = cv2.imread(filepath)
                        face = detector(img, 1)
                        for i, d in enumerate(face):
                            x1 = d.left()
                            y1 = d.top()
                            x2 = d.right()
                            y2 = d.bottom()
                            img = img[y1:y2, x1:x2]
                        lpath = 'D:\\NianXiang_File\\CNN\\Dataset\\small_dataset\\NEW'
                        filename = lpath + '\\' + str(index) + '\\' + str(count) + '.jpg'
                        cv2.imwrite(filename, img)
                        try:
                            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                            resized = cv2.resize(img, (224, 224), cv2.INTER_LANCZOS4)
                            y.append(index)
                            x.append(resized)
                            count = count + 1
                        except:
                            pass

                            # file = Data.static_load_image(ffile)
        np.save('{}_X.npy'.format(file_name), x)
        np.save('{}_Y.npy'.format(file_name), y)
        print("Complete")

class Evaluate(object):
    # 評估模型準確率
    @classmethod
    def evaluate(self, model, x=None, y=None):
        loss, score = model.evaluate(x=x, y=y, batch_size=16, verbose=2)
        print("Accuracy：{:.2f}%".format(score * 100))
    # 預測資料
    @classmethod
    def predict(self, model=None, x=None):
        predict = model.predict(x=x, batch_size=16, verbose=2)
        return predict
    # 混淆矩陣
    @classmethod
    def confusion_matrix(self, model=None, normalize=False, title='Confusion Matrix', cls='le', x=None, y=None):
        predict = model.predict(x=x, batch_size=16, verbose=2)
        pred = np.argmax(predict, axis=1)
        print(np.unique(np.argmax(y, axis=1)), np.unique(pred))
        cnf_matrix = confusion_matrix(np.argmax(y, axis=1), pred)
        np.set_printoptions(precision=2)
        plt.figure()
        if cls == 'be':
            class_name = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
        if cls == 'le':
            class_name = ["Frustration", "Confused", "Bored", "Delightful", "Flow", "Surprise"]
        Evaluate.__plot_confusion_matrix(cnf_matrix, classes=class_name,
                                title=title,
                                normalize=normalize)
        plt.show()

    @classmethod
    def __plot_confusion_matrix(self, cm, classes,
                                normalize=False,
                                title='Confusion matrix',
                                cmap=plt.cm.Blues):
        """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
        """
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            print("Normalized confusion matrix")
        else:
            print('Confusion matrix, without normalization')
        print(cm)

        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)

        fmt = '.2f' if normalize else 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')